In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd

data = pd.read_csv('/content/drive/MyDrive/SKT_FLY_AI/Project/AI_Model/Similarity/law_final.csv', encoding='UTF-8')
data.head(10)

,fact_one,caseNm,courtNm,judmnAdjuDe,caseNo
0,[1] 채무자가 채권자에 대한 기존채무의 변제를 위하여 소비대차계약을 체결하고 강제...,사해행위취소,대법원,2015.10.29.,2012다14975
1,"[1] 계약당사자가 계약내용을 처분문서인 서면으로 작성한 경우, 의사표시의 해석 방...",손해배상(기),대법원,2015.10.15.,2012다64253
2,[1] 위임계약의 일방 당사자가 타방 당사자의 채무불이행을 이유로 위임계약을 해지한...,손해배상(기),대법원,2015.12.23.,2012다71411
3,[1] 임대차 목적물이 화재 등으로 소멸됨으로써 임차인의 목적물 반환의무가 이행불능...,손해배상(기)·손해배상(기),대법원,2017.5.18.,2012다86895
4,[1] 책임감리업무를 수행하는 감리자가 시공 전 설계도서에 기술적인 문제가 있는지 ...,손해배상(기)등,대법원,2015.2.26.,2012다89320
5,[1] 노동조합 및 노동관계조정법 에서 규정하는 사용자의 직장폐쇄가 정당한 쟁의행위...,임금,대법원,2017.4.7.,2013다101425
6,부동산 실권리자명의 등기에 관한 법률을 위반하여 무효인 명의신탁약정에 따라 명의수탁...,소유권이전등기,대법원,2019.6.20.,2013다218156
7,[1] 근로계약 체결에 관한 당사자들의 의사표시에 무효 또는 취소의 사유가 있음을 ...,임금·손해배상(기),대법원,2017.12.22.,2013다25194
8,[1] 혈액제제 제조업체에게 필요한 최선의 조치를 다하여 혈액제제를 통한 감염의 위...,손해배상(의)·손해배상(의)·손해배상(의)·손해배상(의),대법원,2017.11.9.,2013다26708
9,[1] 공무원의 직무상 의무가 순전히 행정기관 내부의 질서를 유지하기 위한 것이거나...,손해배상(기),대법원,2015.5.28.,2013다41431


In [6]:
data.shape

(9563, 5)

In [7]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 42.1 MB/s eta 0:00:00


In [12]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Assuming 'data' is your DataFrame
# Replace NaN values in 'fact_one' column with an empty string
data['fact_one'].fillna("", inplace=True)

# 사용자 입력 텍스트 예시
user_input_text = "오피스텔 임대차 계약서를 위조하고 계약금을 수령한 후 연락이 두절되어 피해 회복을 위해 법원에 소송 제기"

# TF-IDF 벡터라이저 초기화 및 학습
tfidf_vectorizer = TfidfVectorizer()
# Ensure the user_input_text is wrapped in a list to treat it as a single document
tfidf_matrix = tfidf_vectorizer.fit_transform(data['fact_one'].append(pd.Series([user_input_text])))

# 코사인 유사도 계산
cosine_sim = cosine_similarity(tfidf_matrix[-1:], tfidf_matrix[:-1])

# 상위 3개의 가장 유사한 판례의 인덱스 찾기
top_3_indices = cosine_sim.argsort()[0][-3:][::-1]

# 상위 3개의 가장 유사한 판례의 caseNo, 코사인 유사도 및 fact_one 출력
for index in top_3_indices:
    case_no = data.iloc[index]['caseNo']
    cosine_similarity_value = cosine_sim[0][index]
    fact_one = data.iloc[index]['fact_one']
    print(f"caseNo: {case_no}, 코사인 유사도: {cosine_similarity_value}")
    print(f"fact_one: {fact_one}\n")


<ipython-input-12-2cf642f0486e>:15: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tfidf_matrix = tfidf_vectorizer.fit_transform(data['fact_one'].append(pd.Series([user_input_text])))


caseNo: 2022누49221, 코사인 유사도: 0.11907591614962133
fact_one: 갑이 소유하던 주택을 타인에게 양도한 후 사망하자 배우자 을이 이를 1세대 1주택으로 양도소득세를 신고·납부하였는데, 위 주택 양도 당시 을이 자신 소유의 오피스텔 2채를 모두 주거용으로 사용(임대)하였다는 이유로 관할 세무서장이 그와 1세대를 구성하던 갑도 1세대 3주택자에 해당한다고 보아 납세의무 승계인인 을에게 양도소득세를 경정·고지한 사안에서, 을 소유의 오피스텔 모두 공부상의 용도에 따라 ‘업무시설(오피스텔)’로 보아야 하므로, 위 주택 양도는 ‘1세대 1주택’의 양도에 해당한다고 한 사례

caseNo: 2022누49221, 코사인 유사도: 0.11907591614962133
fact_one: 갑이 소유하던 주택을 타인에게 양도한 후 사망하자 배우자 을이 이를 1세대 1주택으로 양도소득세를 신고·납부하였는데, 위 주택 양도 당시 을이 자신 소유의 오피스텔 2채를 모두 주거용으로 사용(임대)하였다는 이유로 관할 세무서장이 그와 1세대를 구성하던 갑도 1세대 3주택자에 해당한다고 보아 납세의무 승계인인 을에게 양도소득세를 경정·고지한 사안에서, 을 소유의 오피스텔 모두 공부상의 용도에 따라 ‘업무시설(오피스텔)’로 보아야 하므로, 위 주택 양도는 ‘1세대 1주택’의 양도에 해당한다고 한 사례

caseNo: 2019도9773, 코사인 유사도: 0.1026025758586823
fact_one: [1] 횡령죄의 객체인 타인의 재물이 누구의 소유인지가 횡령죄의 성립에 영향을 미치는지 여부(소극) / 주식회사의 주주나 대표이사 또는 그에 준하여 회사 자금의 보관이나 운용에 관한 사실상의 사무를 처리하는 자가 회사 소유의 재산을 사적인 용도로 함부로 처분한 경우, 횡령죄가 성립하는지 여부(적극)[2] 피고인들이 공모하여 갑 주식회사 등 피해 회사가 납품하는 물품을 마치 피해 회사의 자회사로서 서류상으로만 존재하는 을 주식회사 등이 납품하는 것처럼 서류를 꾸

In [13]:
import pandas as pd

# Read the original dataset that contains the complete set of columns
original_data = pd.read_csv('/content/drive/MyDrive/SKT_FLY_AI/Project/AI_Model/Similarity/law_final.csv', encoding='UTF-8')

In [14]:
# Assuming 'caseNo' is unique and can be used as a key for merging
columns_to_add = original_data[['caseNo', 'caseNm', 'courtNm', 'judmnAdjuDe']]

In [16]:
# Merge the extracted columns back into your data DataFrame
data_final = pd.merge(data, columns_to_add, on='caseNo', how='left')

In [17]:
# Display the first few rows of the merged DataFrame
print(data_final.head(10))

                                            fact_one         caseNm_x  \
0  [1] 채무자가 채권자에 대한 기존채무의 변제를 위하여 소비대차계약을 체결하고 강제...           사해행위취소   
1  [1] 채무자가 채권자에 대한 기존채무의 변제를 위하여 소비대차계약을 체결하고 강제...           사해행위취소   
2  [1] 계약당사자가 계약내용을 처분문서인 서면으로 작성한 경우, 의사표시의 해석 방...          손해배상(기)   
3  [1] 계약당사자가 계약내용을 처분문서인 서면으로 작성한 경우, 의사표시의 해석 방...          손해배상(기)   
4  [1] 위임계약의 일방 당사자가 타방 당사자의 채무불이행을 이유로 위임계약을 해지한...          손해배상(기)   
5  [1] 위임계약의 일방 당사자가 타방 당사자의 채무불이행을 이유로 위임계약을 해지한...          손해배상(기)   
6  [1] 임대차 목적물이 화재 등으로 소멸됨으로써 임차인의 목적물 반환의무가 이행불능...  손해배상(기)·손해배상(기)   
7  [1] 임대차 목적물이 화재 등으로 소멸됨으로써 임차인의 목적물 반환의무가 이행불능...  손해배상(기)·손해배상(기)   
8  [1] 책임감리업무를 수행하는 감리자가 시공 전 설계도서에 기술적인 문제가 있는지 ...         손해배상(기)등   
9  [1] 책임감리업무를 수행하는 감리자가 시공 전 설계도서에 기술적인 문제가 있는지 ...         손해배상(기)등   

  courtNm_x judmnAdjuDe_x      caseNo         caseNm_y courtNm_y judmnAdjuDe_y  
0       대법원   2015.10.29.  2012다14975           사해행위취소       대법원   2015.10.29.  
1       대법원   2015.10.29.  2012다14